# Build a tile index (`tindex`) for a set of DPS output

In [ ]:
from maap.maap import MAAP
maap = MAAP()

In [ ]:
ICESAT2_BOREAL_REPO_PATH = '/projects/code/icesat2_boreal'     
ICESAT2_BOREAL_LIB_PATH = ICESAT2_BOREAL_REPO_PATH + '/lib'

In [ ]:
import geopandas as gpd
import pandas as pd
import os

import sys
import s3fs

sys.path.append(ICESAT2_BOREAL_LIB_PATH)

print("Importing packages complete.")

In [ ]:
!pip install cogeo_mosaic

In [ ]:
import importlib
import mosaiclib
import maplib_folium
importlib.reload(mosaiclib)
importlib.reload(maplib_folium)

In [ ]:
import mosaiclib
from mosaiclib import *

## Build new tindex and MosiacJSONs

In [ ]:
DICT_BUILD_TINDEX_ATL08_FILT = {
  'SET': 'ATL08',
 'USER': 'montesano',
 'ALG_NAME': 'run_extract_atl08_covars', 
 'ALG_VERSION': 'extract_atl08_covars',
 'VAR': 'ATL08_filt_extract',
 'BATCH_NAME': '2020',
 'YEAR': 2024,
 'DPS_MONTH_LIST': '02 03',
 'DPS_DAY_MIN': 1,
 'TILES_INDEX_PATH': '/projects/shared-buckets/montesano/databank/boreal_tiles_v004_model_ready.gpkg'
}

In [ ]:
DICT_BUILD_TINDEX_AGB = {
    'SET' : 'BOREAL_MAP',
    'USER' : 'lduncanson',
    'ALG_NAME' : 'run_boreal_biomass_map',
    'ALG_VERSION' : 'boreal_agb_2024_v6', 
    'VAR' : 'AGB',
    'BATCH_NAME' : 'AGB_H30_2020/Version2_SD',
    'YEAR': 2024,
    'DPS_MONTH_LIST' : '07 08 09 10',        
    'DPS_DAY_MIN' : 1 ,
    'TILES_INDEX_PATH': boreal_tile_index_path
}
DICT_BUILD_TINDEX_HT = {
    'SET' : 'BOREAL_MAP_CHECK',
    'USER' : 'lduncanson',
    'ALG_NAME' : 'run_boreal_biomass_map',
    'ALG_VERSION' : 'boreal_agb_2024_v6', 
    'VAR' : 'HT',
    'BATCH_NAME' : 'Ht_H30_2020/Version2_SD',
    'YEAR': 2024,
    'DPS_MONTH_LIST' : '09 10', #'07 08 09 10',        
    'DPS_DAY_MIN' : 1 ,
    'TILES_INDEX_PATH': boreal_tile_index_path
}

In [ ]:
DICT_BUILD_TINDEX = DICT_BUILD_TINDEX_ATL08_FILT
DICT_BUILD_TINDEX = DICT_BUILD_TINDEX_HT

In [ ]:
%%time
if True:
    DPS_IDENTIFIER = f"{DICT_BUILD_TINDEX['ALG_VERSION']}/{DICT_BUILD_TINDEX['BATCH_NAME']}"

    OUTDIR = f"/projects/my-public-bucket/DPS_tile_lists/{DICT_BUILD_TINDEX['SET']}/{DPS_IDENTIFIER}"
    !mkdir -p $OUTDIR

    # Build tindex
    args = f"--RETURN_DUPS --user {DICT_BUILD_TINDEX['USER']} --dps_identifier {DPS_IDENTIFIER} \
    -alg_name {DICT_BUILD_TINDEX['ALG_NAME']} -t {DICT_BUILD_TINDEX['VAR']} -y {DICT_BUILD_TINDEX['YEAR']} \
    --dps_month_list {DICT_BUILD_TINDEX['DPS_MONTH_LIST']} -d_min {DICT_BUILD_TINDEX['DPS_DAY_MIN']} --outdir {OUTDIR} \
    -boreal_tile_index_path {DICT_BUILD_TINDEX['TILES_INDEX_PATH']}"

    !time /projects/env/above/bin/python /projects/code/icesat2_boreal/lib/build_tindex_master.py $args --WRITE_TINDEX_MATCHES_GDF

### Check the tindex

In [ ]:
fn_ht  = 's3://maap-ops-workspace/shared/montesano/DPS_tile_lists/BOREAL_MAP/boreal_agb_2024_v6/Ht_H30_2020/Version2_SD/HT_tindex_master.csv' #os.path.join(OUTDIR,"HT_tindex_master.csv")
fn_agb = 's3://maap-ops-workspace/shared/montesano/DPS_tile_lists/BOREAL_MAP/boreal_agb_2024_v6/AGB_H30_2020/Version2_SD/AGB_tindex_master.csv'

In [ ]:
tindex_ht = pd.read_csv(fn_ht)
tindex_agb = pd.read_csv(fn_agb)

In [ ]:
tindex_ht.shape

In [ ]:
tindex_agb.shape

In [ ]:
boreal_tiles = gpd.read_file(boreal_tile_index_path.replace('.gpkg','_model_ready.gpkg'))
boreal_tiles_orig = gpd.read_file(boreal_tile_index_path)

In [ ]:
MISSING_LIST_AGB = boreal_tiles[~boreal_tiles.tile_num.isin(tindex_agb.tile_num.to_list())].tile_num.to_list()
MISSING_LIST_AGB

In [ ]:
MISSING_LIST_HT = boreal_tiles[~boreal_tiles.tile_num.isin(tindex_ht.tile_num.to_list())].tile_num.to_list()
MISSING_LIST_HT

In [ ]:
ATL08_FILT_TINDEX_FN_DICT['2020']

In [ ]:
tindex_atl08 = pd.read_csv(ATL08_FILT_TINDEX_FN_DICT['2020'])
ATL08_TILES = tindex_atl08.tile_num.to_list()
len(ATL08_TILES)

In [ ]:
tindex_atl08[tindex_atl08.tile_num.isin([23497, 23498, 23499])]

In [ ]:
tindex_agb[tindex_agb.tile_num == 22937].s3_path.to_list()[0]

In [ ]:
#boreal_tiles.explore(m=boreal_tiles_orig[boreal_tiles_orig.tile_num.isin(NEED_LIST)].explore(color='red'))

In [ ]:
s3 = s3fs.S3FileSystem(anon=True)

In [ ]:
s3.download(tindex_agb[tindex_agb.tile_num == 3082].s3_path.to_list()[0], '/projects/my-public-bucket/tmp')